# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../WeatherPy/Outputs/city_data.csv")
city_data


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Maniwaki,46.38,-75.97,42.48,80,54,9.75,CA,1602376861
1,Guerrero Negro,27.98,-114.06,70.52,80,0,17.47,MX,1602376996
2,Comodoro Rivadavia,-45.87,-67.50,51.80,62,0,3.36,AR,1602376942
3,Egvekinot,66.32,-179.17,31.17,93,69,15.17,RU,1602376996
4,Hailar,49.20,119.70,33.15,89,100,20.29,CN,1602376997
...,...,...,...,...,...,...,...,...,...
657,Marrakesh,31.63,-8.01,75.20,47,20,2.24,MA,1602377153
658,Abomey,7.18,1.99,72.10,98,91,4.12,BJ,1602377154
659,Havøysund,71.00,24.66,44.60,100,100,16.11,NO,1602377154
660,Mīzan Teferī,6.98,35.58,59.90,91,69,1.99,ET,1602377154


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# taken from banking_desert activity
# configure gmaps with API key
gmaps.configure(api_key =g_key)

# Store lat/long into locations
locations = city_data[['Latitude', 'Longitude']]

# Store humidity as weight
weight = city_data['Humidity']

In [27]:
# Plot heatmap
fig = gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weight, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Narrow cities with A max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph,Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

weather_criteria_df = city_data.loc[(city_data['Wind Speed']<=10) & (city_data['Cloudiness']==0) & \
                                    (city_data['Temperature']>=70) & (city_data['Temperature']<=80)]


len(weather_criteria_df)

20

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# Store into variable named hotel_df.
hotel_df = weather_criteria_df.loc[:,['City','Country','Latitude','Longitude']]
hotel_df.head(2)

,City,Country,Latitude,Longitude
74,Gwadar,PK,25.12,62.33
81,Parīchhatgarh,IN,28.98,77.93


In [49]:
# create one request to see what is returned
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel1_lat_long= "25.12, 62.33"

params1 = {"location": hotel1_lat_long, "type" : "lodging","radius" : 5000,"keyword" : "hotel","key" : g_key}

one_hotel = requests.get(url, params=params1)
one_response=one_hotel.json()
print(json.dumps(one_response, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 25.1039117,
                    "lng": 62.32866559999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.10479517989272,
                        "lng": 62.32991492989272
                    },
                    "southwest": {
                        "lat": 25.10209552010727,
                        "lng": 62.32721527010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Zaver Pearl Continental Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1068,
                    "html_attributions": [
          

In [50]:
print(one_response['results'][0]['name'])

Zaver Pearl Continental Hotel


In [48]:
# Add a "Hotel Name" column to the DataFrame (going to create and empty list and then add into dataframe from there)

# need to create variable to get query lat/long to pull correctly
lat = hotel_df.loc[:,'Latitude']
long = hotel_df.loc[:,'Longitude']
lat_long = (f"{lat},{long}")


In [ ]:
# Set parameters to search for hotels with 5000 meters.
# sample query url: https://maps.googleapis.com/maps/api/place/nearbysearch/json?
# location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"location": lat_long, "type" : "lodging","radius" : 5000,"keyword" : "hotel","key" : g_key}

# Loop through hotel_df to grab nearest hotel names
hotels = []



In [ ]:
hotel_df['Hotel Name'] = hotels

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
